<p style="font-family: Arial, sans-serif; color: yellow; font-size: 18px;">This is a notebook for initial analysis of raw data having >1000 variables (hence the name). We want to find out the kind of values that exist for each variable, the degree of missingness and whether there is any statistical variability in the values. Using this we could select our top 100 variables for a deeper EDA along with creating some derived variables that could be of importance to Causality.</p>

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import math
from collections import Counter

In [12]:

DATA_PATH = Path("D:/DS/IPE Global/ML/lbw/data/AI_Cohort_LabeledData.csv")
DICT_PATH = Path("D:/DS/IPE Global/ML/lbw/data/Data dictionary_AI_Cohort.xlsx")
OUTPUT_XLSX = Path("D:/DS/IPE Global/ML/lbw/data/eda_profile.xlsx")
OUTPUT_SUMMARY_CSV = Path("D:/DS/IPE Global/ML/lbw/data/eda_profile_summary.csv")
OUTPUT_TOPVALUES_CSV = Path("D:/DS/IPE Global/ML/lbw/data/eda_profile_top_values.csv")
OUTPUT_TYPEMAP_CSV = Path("D:/DS/IPE Global/ML/lbw/data/eda_profile_type_map.csv")

In [13]:
from charset_normalizer import from_path
best = from_path(DATA_PATH).best()

In [14]:
best.encoding

'utf_8'

In [15]:
DATA_PATH

WindowsPath('D:/DS/IPE Global/ML/lbw/data/AI_Cohort_LabeledData.csv')

In [16]:

def _first_present(cols, candidates):
    for c in candidates:
        if c in cols:
            return c
    return None

def load_dictionary(dict_path: Path) -> pd.DataFrame:
    xls = pd.ExcelFile(dict_path)
    frames = []
    for s in xls.sheet_names:
        df = pd.read_excel(dict_path, sheet_name=s)
        df.columns = [str(c).strip() for c in df.columns]
        cols = df.columns.tolist()
        field_col = _first_present(cols, ['Field Label','Variable','Name','Field','Column','Column Name'])
        type_col = None
        for c in cols:
            if 'type' in c.lower():
                type_col = c
                break
        desc_col = _first_present(cols, ['Description','Variable Label','Label'])
        if field_col is None:
            continue
        out = pd.DataFrame({
            'sheet': s,
            'field': df[field_col].astype(str).str.strip(),
            'declared_type': df[type_col].astype(str).str.strip() if type_col else pd.Series([np.nan]*len(df)),
            'description': df[desc_col].astype(str).str.strip() if desc_col else pd.Series([np.nan]*len(df))
        })
        frames.append(out)
    if not frames:
        return pd.DataFrame(columns=['sheet','field','declared_type','description'])
    di = pd.concat(frames, ignore_index=True).dropna(subset=['field'])
    di['field_lower'] = di['field'].str.lower()
    return di

def normalize_type(t: str) -> str:
    if not isinstance(t, str):
        return 'unknown'
    tl = t.strip().lower()
    if any(k in tl for k in ['int', 'integer']):
        return 'integer'
    if any(k in tl for k in ['float','double','decimal','numeric','number']):
        return 'float'
    if any(k in tl for k in ['bool','yes/no']):
        return 'boolean'
    if any(k in tl for k in ['date','datetime','timestamp','time']):
        return 'date'
    if any(k in tl for k in ['categor','enum','choice','select','code','label']):
        return 'categorical'
    if any(k in tl for k in ['text','string','free','remarks','note']):
        return 'text'
    return 'unknown'

def build_type_map(dictionary_df: pd.DataFrame, df_cols: list) -> pd.DataFrame:
    df_cols_lower = [c.lower() for c in df_cols]
    di = dictionary_df.copy()
    di_norm = di.groupby('field_lower', as_index=False).agg({
        'sheet':'first',
        'field':'first',
        'declared_type':'first',
        'description':'first'
    })
    di_norm['normalized_type'] = di_norm['declared_type'].apply(normalize_type)
    di_map = {r['field_lower']: r for _, r in di_norm.iterrows()}
    rows = []
    for col, col_l in zip(df_cols, df_cols_lower):
        if col_l in di_map:
            r = di_map[col_l]
            rows.append({
                'column': col,
                'sheet': r['sheet'],
                'declared_type_raw': r['declared_type'],
                'declared_type': r['normalized_type'],
                'description': r['description']
            })
        else:
            rows.append({
                'column': col,
                'sheet': '',
                'declared_type_raw': '',
                'declared_type': 'unknown',
                'description': ''
            })
    return pd.DataFrame(rows)

def coerce_series(s: pd.Series, kind: str) -> pd.Series:
    if kind == 'integer':
        return pd.to_numeric(s, errors='coerce')
    if kind == 'float':
        return pd.to_numeric(s, errors='coerce')
    if kind == 'boolean':
        m = {
            'true': True, 'false': False, 'yes': True, 'no': False, 'y': True, 'n': False,
            '1': True, '0': False, 1: True, 0: False, 't': True, 'f': False
        }
        return s.map(lambda x: m.get(str(x).strip().lower(), np.nan))
    if kind == 'date':
        return pd.to_datetime(s, errors='coerce')
    return s

def coerce_df(df: pd.DataFrame, type_map: pd.DataFrame) -> pd.DataFrame:
    coerced = df.copy()
    kind_by_col = dict(zip(type_map['column'], type_map['declared_type']))
    for c in coerced.columns:
        kind = kind_by_col.get(c, 'unknown')
        if kind in ('integer','float','boolean','date','categorical','text'):
            coerced[c] = coerce_series(coerced[c], kind)
    return coerced

def tukey_bounds(x: pd.Series):
    q1 = np.nanpercentile(x, 25)
    q3 = np.nanpercentile(x, 75)
    iqr = q3 - q1
    lb = q1 - 1.5 * iqr
    ub = q3 + 1.5 * iqr
    return lb, ub, q1, q3, iqr

def numeric_stats(x: pd.Series):
    arr = pd.to_numeric(x, errors='coerce')
    n = len(arr)
    miss = arr.isna().sum()
    nonmiss = n - miss
    desc = {
        'count_non_missing': int(nonmiss),
        'count_missing': int(miss),
        'pct_missing': float((miss / n * 100)) if n else np.nan,
        'n_zeros': int((arr == 0).sum(skipna=True)),
        'n_negatives': int((arr < 0).sum(skipna=True)),
        'unique_non_missing': int(arr.nunique(dropna=True)),
        'min': float(np.nanmin(arr)) if nonmiss else np.nan,
        'p01': float(np.nanpercentile(arr, 1)) if nonmiss else np.nan,
        'p05': float(np.nanpercentile(arr, 5)) if nonmiss else np.nan,
        'p25': float(np.nanpercentile(arr, 25)) if nonmiss else np.nan,
        'p50': float(np.nanpercentile(arr, 50)) if nonmiss else np.nan,
        'p75': float(np.nanpercentile(arr, 75)) if nonmiss else np.nan,
        'p95': float(np.nanpercentile(arr, 95)) if nonmiss else np.nan,
        'p99': float(np.nanpercentile(arr, 99)) if nonmiss else np.nan,
        'max': float(np.nanmax(arr)) if nonmiss else np.nan,
        'mean': float(np.nanmean(arr)) if nonmiss else np.nan,
        'std': float(np.nanstd(arr)) if nonmiss else np.nan,
    }
    if nonmiss:
        lb, ub, q1, q3, iqr = tukey_bounds(arr.values)
        desc.update({
            'iqr': float(iqr),
            'tukey_lb': float(lb),
            'tukey_ub': float(ub),
            'lower_outlier_rate_%': float((arr < lb).mean() * 100),
            'upper_outlier_rate_%': float((arr > ub).mean() * 100)
        })
    else:
        desc.update({
            'iqr': np.nan, 'tukey_lb': np.nan, 'tukey_ub': np.nan,
            'lower_outlier_rate_%': np.nan, 'upper_outlier_rate_%': np.nan
        })
    return desc

def categorical_stats(x: pd.Series, top_k=20):
    s = x.astype('string')
    n = len(s)
    miss = s.isna().sum()
    nonmiss = n - miss
    vc = s.value_counts(dropna=True)
    unique = int(vc.shape[0])
    top = vc.head(top_k)
    mode = top.index[0] if unique>0 else np.nan
    mode_freq = int(top.iloc[0]) if unique>0 else 0
    mode_prop = (mode_freq / nonmiss * 100) if nonmiss and unique>0 else np.nan
    p = (vc / nonmiss).values if nonmiss else np.array([])
    entropy = float(-np.sum(p * np.log2(p))) if len(p) else np.nan
    stats = {
        'count_non_missing': int(nonmiss),
        'count_missing': int(miss),
        'pct_missing': float((miss / n * 100)) if n else np.nan,
        'unique_non_missing': unique,
        'mode': mode,
        'mode_freq': mode_freq,
        'mode_prop_%': float(mode_prop) if not math.isnan(mode_prop) else np.nan,
        'entropy_bits': float(entropy) if not math.isnan(entropy) else np.nan
    }
    top_table = pd.DataFrame({
        'value': top.index.astype(str),
        'count': top.values
    })
    return stats, top_table

def date_stats(x: pd.Series):
    s = pd.to_datetime(x, errors='coerce')
    n = len(s)
    miss = s.isna().sum()
    nonmiss = n - miss
    stats = {
        'count_non_missing': int(nonmiss),
        'count_missing': int(miss),
        'pct_missing': float((miss / n * 100)) if n else np.nan,
        'min_date': s.min(),
        'p25_date': s.quantile(0.25),
        'median_date': s.quantile(0.5),
        'p75_date': s.quantile(0.75),
        'max_date': s.max()
    }
    return stats

def profile_dataframe(df: pd.DataFrame, type_map: pd.DataFrame, top_k=20):
    type_lookup = dict(zip(type_map['column'], type_map['declared_type']))
    desc_lookup = dict(zip(type_map['column'], type_map.get('description', pd.Series(dtype=str))))
    sheet_lookup = dict(zip(type_map['column'], type_map.get('sheet', pd.Series(dtype=str))))
    rows = []
    top_values_rows = []
    N = len(df)
    for col in df.columns:
        declared = type_lookup.get(col, 'unknown')
        desc = desc_lookup.get(col, '')
        sheet = sheet_lookup.get(col, '')
        base = {
            'column': col,
            'sheet': sheet,
            'declared_type': declared,
            'description': desc,
            'n_rows': N
        }
        s = df[col]
        if declared in ('integer','float') or (pd.api.types.is_numeric_dtype(s) and declared != 'categorical'):
            stats = numeric_stats(s)
            base.update(stats)
        elif declared == 'date' or pd.api.types.is_datetime64_any_dtype(s):
            stats = date_stats(s)
            base.update(stats)
        else:
            stats, top_table = categorical_stats(s, top_k=20)
            base.update(stats)
            top_table.insert(0, 'column', col)
            top_values_rows.append(top_table)
        rows.append(base)
    summary = pd.DataFrame(rows)
    top_values = pd.concat(top_values_rows, ignore_index=True) if top_values_rows else pd.DataFrame(columns=['column','value','count'])
    summary['is_constant'] = (summary['unique_non_missing'].fillna(0) <= 1)
    summary['high_cardinality'] = summary['unique_non_missing'] > 100
    summary = summary.sort_values(['pct_missing','unique_non_missing'], ascending=[True, True])
    return summary, top_values


In [17]:

# Load inputs
df = pd.read_csv(DATA_PATH, encoding=best.encoding)

C:\Users\Dell\AppData\Local\Temp\ipykernel_21852\2039903953.py:2: DtypeWarning: Columns (9,31,35,46,92,96,98,101,104,107,110,111,137,138,139,140,141,142,143,144,145,146,147,149,155,159,160,166,170,171,172,173,241,296,298,370,371,372,373,374,380,487,504,505,507,508,509,510,511,512,513,514,515,516,571,584,585,595,596,597,598,601,951,953,955,957,959,961,990,991,992,993,1005,1009,1013,1014,1015,1021,1022,1023,1024,1040,1041,1042,1044,1045,1046,1048,1049,1050,1051,1052,1053,1054,1055,1056,1057,1058,1059,1061,1062,1063,1064,1065,1066,1067,1068,1069,1070,1075,1076,1077,1133,1156,1158,1159,1208,1209,1210,1211,1219,1221,1249,1251,1306,1307,1339,1377,1378,1391,1392) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(DATA_PATH, encoding=best.encoding)


In [18]:
df.shape

(37353, 1396)

In [19]:

dictionary_df = load_dictionary(DICT_PATH)
type_map = build_type_map(dictionary_df, df.columns.tolist())

In [20]:

# Infer types for 'unknown' columns (simple heuristic)
infer_rows = []
for c in type_map.loc[type_map['declared_type']=='unknown', 'column']:
    s = df[c]
    kind = 'unknown'
    if pd.api.types.is_integer_dtype(s) or pd.api.types.is_float_dtype(s):
        kind = 'float'
    else:
        s_num = pd.to_numeric(s, errors='coerce')
        if s_num.notna().mean() > 0.9:
            kind = 'float'
        else:
            s_dt = pd.to_datetime(s, errors='coerce')
            if s_dt.notna().mean() > 0.8:
                kind = 'date'
            else:
                nu = s.astype(str).nunique(dropna=True)
                kind = 'categorical' if nu < 100 else 'text'
    infer_rows.append((c, kind))


C:\Users\Dell\AppData\Local\Temp\ipykernel_21852\378956463.py:13: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  s_dt = pd.to_datetime(s, errors='coerce')
C:\Users\Dell\AppData\Local\Temp\ipykernel_21852\378956463.py:13: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  s_dt = pd.to_datetime(s, errors='coerce')
C:\Users\Dell\AppData\Local\Temp\ipykernel_21852\378956463.py:13: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  s_dt = pd.to_datetime(s, errors='coerce')
C:\Users\Dell\AppData\Local\Temp\ipykernel_21852\378956463.py:13: UserWarning: Could not infer format, so each element

In [21]:

for col, kind in infer_rows:
    type_map.loc[type_map['column']==col, 'declared_type'] = kind


In [22]:

# Coerce and profile
df_coerced = coerce_df(df, type_map)
summary_df, top_values_df = profile_dataframe(df_coerced, type_map, top_k=20)


C:\Users\Dell\AppData\Local\Temp\ipykernel_21852\3801065634.py:98: UserWarning: Parsing dates in %d-%m-%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  return pd.to_datetime(s, errors='coerce')
C:\Users\Dell\AppData\Local\Temp\ipykernel_21852\3801065634.py:98: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  return pd.to_datetime(s, errors='coerce')
C:\Users\Dell\AppData\Local\Temp\ipykernel_21852\3801065634.py:98: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  return pd.to_datetime(s, errors='coerce')
C:\Users\Dell\AppData\Local\Temp\ipykernel_21852\3801065634.py:98: UserWarning: Could not infer format, so each element will be pars

In [23]:

# Save outputs
summary_df.to_csv(OUTPUT_SUMMARY_CSV, index=False)
top_values_df.to_csv(OUTPUT_TOPVALUES_CSV, index=False)
type_map.to_csv(OUTPUT_TYPEMAP_CSV, index=False)
with pd.ExcelWriter(OUTPUT_XLSX, engine='xlsxwriter') as writer:
    summary_df.to_excel(writer, sheet_name='Summary', index=False)
    top_values_df.to_excel(writer, sheet_name='TopValues', index=False)
    type_map.to_excel(writer, sheet_name='TypeMap', index=False)